In [1]:
import pandas as pd
import re

### Load Stata Source

In [2]:
stata_file = open( "data/QprExport-2018.06.29.stata", 'r' )
stata_list = [line.strip() for line in stata_file.readlines() ]

In [3]:
stata_list[ 0 ]

'********************************************************************************'

### Load Pirl 'n' Parquet Names

In [4]:
pirl_to_parquet = pd.read_csv( "data/raw-pirl-to-parquet-names.csv", usecols=[ "PIRL_ElementNumber", "ParquetName" ] )
pirl_to_parquet[ "Pirl" ] = "PIRL" + pirl_to_parquet.PIRL_ElementNumber
pirl_to_parquet.drop( "PIRL_ElementNumber", inplace=True, axis=1 )
    
print( pirl_to_parquet.head() )

# get as dictionary: EZ!
pirl_dictionary = dict( zip( pirl_to_parquet.Pirl, pirl_to_parquet.ParquetName ) )

                      ParquetName     Pirl
0                       ObsNumber    PIRL0
1  UniqueIndividualIdentifierWIOA  PIRL100
2        StateCodeOfResidenceWIOA  PIRL101
3           CountyCodeOfResidence  PIRL102
4              ZipCodeOfResidence  PIRL103


In [5]:
pirl_dictionary

{'PIRL0': 'ObsNumber',
 'PIRL100': 'UniqueIndividualIdentifierWIOA',
 'PIRL1000': 'DateOfFirstBasicCareerServiceSelfService',
 'PIRL1001': 'DateOfFirstBasicCareerServiceStaffAssisted',
 'PIRL1002': 'MostRecentDateReceivedBasicCareerServicesSelfServi',
 'PIRL1003': 'MostRecentDateReceivedBasicCareerServicesStaffAssi',
 'PIRL1004': 'DateOfMostRecentCareerServiceWIOA',
 'PIRL1005': 'MostRecentDateReceivedStaffAssistedServicesDVOPSpe',
 'PIRL1006': 'DateReferredToDepartmentOfVeteransAffairsVocationa',
 'PIRL1007': 'DateOfMostRecentReportableIndividualContact',
 'PIRL101': 'StateCodeOfResidenceWIOA',
 'PIRL102': 'CountyCodeOfResidence',
 'PIRL103': 'ZipCodeOfResidence',
 'PIRL104': 'EconomicLaborMarketAreaAndPhysicalLocationCode',
 'PIRL105': 'SpecialProjectId1',
 'PIRL106': 'SpecialProjectId2',
 'PIRL107': 'SpecialProjectId3',
 'PIRL108-A': 'ETAAssigned1StLocalWorkforceBoardCode',
 'PIRL108-B': 'ETAAssigned2NdLocalWorkforceBoardCode',
 'PIRL108-C': 'ETAAssigned3RdLocalWorkforceBoardCode',


In [6]:
for i in range( 5 ):
    
    print( stata_list[ i ] )

********************************************************************************
*************************QPR TEST AGGREGATION RULES*****************************
********************************************************************************
cd "H:\Data Work\WIPS Test File"
local cdate: display %td-CY-m-D date(c(current_date),"DMY")


### Build Stop List of Translatable Lines

In [7]:
translatable = [ "local", "count", "replace" ]
translatable

['local', 'count', 'replace']

### Kill Multiple Space Chars

In [8]:
def kill_extra_spaces( raw_line ):
    
    # replace multiple spaces w/ just one
    return re.sub( ' +', ' ', raw_line )

kill_extra_spaces( "12  23  45               4 4 4 4 4        45 64 6  ASFD     " )

'12 23 45 4 4 4 4 4 45 64 6 ASFD '

### Normalize Parenthesis

In [9]:
def normalize_parenthesis( raw_line ):
    
    # add spaces between parens and chars
    raw_line = re.sub( r'\(', "( ", re.sub( r'\)', " )", raw_line ) )

    return kill_extra_spaces( raw_line )

normalize_parenthesis( "//         putexcel M21 = (r(N)),    sheet(       `report' ) " )

"// putexcel M21 = ( r( N ) ), sheet( `report' ) "

### Normalize Plus Signs

In [10]:
def normalize_plus_signs( raw_line ):
    
    # add spaces between plus signs and chars
    raw_line = re.sub( r'\+', " + ", raw_line )

    return kill_extra_spaces( raw_line )

normalize_plus_signs( "count if ( ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==2 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==3 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==4 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==5 ) ) & `FS' & `total' & `prevpartdef'" )


"count if ( ( ( PIRL211 + PIRL212 + PIRL213 + PIRL214 + PIRL215 )==2 ) | ( ( PIRL211 + PIRL212 + PIRL213 + PIRL214 + PIRL215 )==3 ) | ( ( PIRL211 + PIRL212 + PIRL213 + PIRL214 + PIRL215 )==4 ) | ( ( PIRL211 + PIRL212 + PIRL213 + PIRL214 + PIRL215 )==5 ) ) & `FS' & `total' & `prevpartdef'"

### Normalize Equals Signs

In [11]:
def normalize_equals( raw_line ):
    
    # add spaces between plus signs and chars
    raw_line = re.sub( r'==', " = ", raw_line )

    return kill_extra_spaces( raw_line )

normalize_equals( "count if ( ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==2 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==3 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==4 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 )==5 ) ) & `FS' & `total' & `prevpartdef'" )

"count if ( ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 ) = 2 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 ) = 3 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 ) = 4 ) | ( ( PIRL211+PIRL212+PIRL213+PIRL214+PIRL215 ) = 5 ) ) & `FS' & `total' & `prevpartdef'"

### Translate Conjunctions

In [12]:
def translate_conjuntions( raw_line ):
    
    # replace | with OR, add spaces between
    raw_line = re.sub( r'\|', " OR ", raw_line )
    
    # replace & with AND, add spaces between
    raw_line = re.sub( r'&', " AND ", raw_line )
    
    return kill_extra_spaces( raw_line )

translate_conjuntions( "replace age = age - 1 if (month(PIRL906) < month(PIRL200)) | (month(PIRL906) == month(PIRL200) & day(PIRL906) < day(PIRL200))" )
    

'replace age = age - 1 if (month(PIRL906) < month(PIRL200)) OR (month(PIRL906) == month(PIRL200) AND day(PIRL906) < day(PIRL200))'

### Translate Variable Declarations

In [13]:
def translate_variable_declarations( raw_line ):
    
    raw_line = re.sub( '^local ([A-Za-z0-9]{1,}) "', r'val \1 = "', raw_line )
    
    return kill_extra_spaces( raw_line )

translate_variable_declarations( 'local Adult "( inlist( PIRL903,1,2,3,4 ) )' )

'val Adult = "( inlist( PIRL903,1,2,3,4 ) )'

### Translate 'IN' clauses

In [14]:
def translate_in_clause( raw_line ):
    
    # WORKS: raw_line = re.sub( r'(PIRL[0-9]{1,3})', r'__1st__\1__1st__', raw_line )
    # WORKS: raw_line = re.sub( r'inlist\(', 'MATCH', raw_line )
    # WORKS: raw_line = re.sub( r'(inlist)', r'--YEP--\1--YEP--', raw_line )
    # WO$RKS: raw_line = re.sub( r'(inlist\() (PIRL[0-9]{1,3},)', r'1st:[\1] 2nd: [\2]', raw_line )
    raw_line = re.sub( r'(inlist\() (PIRL[0-9]{1,3})', r'\2 IN (', raw_line )
    
    # kludgey, but it works: replace " IN (," with " IN ( "
    raw_line = re.sub( r' IN \(,', r' IN ( ', raw_line )
    
    return kill_extra_spaces( raw_line )

translate_in_clause( "( inlist( PIRL903,1,2,3,4 ) )" )    

'( PIRL903 IN ( 1,2,3,4 ) )'

### Translate Date Variable Declarations

In [15]:
# declare month abbreviations to numerics as a dictionary
months = {}
months[ "jan" ] = "01"
months[ "feb" ] = "02"
months[ "mar" ] = "03"
months[ "apr" ] = "04"
months[ "may" ] = "05"
months[ "jun" ] = "06"
months[ "jul" ] = "07"
months[ "aug" ] = "08"
months[ "sep" ] = "09"
months[ "oct" ] = "10"
months[ "nov" ] = "11"
months[ "dec" ] = "12"

def substitute_months( match ):
    
    return 'val {} = "{}-{}-{}"'.format( match.group( 1 ), match.group( 4 ), months[ match.group( 3 ) ], match.group( 2 ) )

def translate_date_vars( raw_line ):
    
    # This line calls a function w/ the match object.  ¡Genial!
    # Here: https://stackoverflow.com/questions/17136127/calling-a-function-on-captured-group-in-re-sub
    raw_line = re.sub( r'local ([a-zA-Z0-9]{1,}) (\d\d)([a-zA-Z]{3})(\d\d\d\d)', substitute_months, raw_line )
    
    # Plain, w/o month abbreviation to numeric lookup
    #raw_line = re.sub( r'local ([a-zA-Z]{1,}) (\d\d)([a-zA-Z]{3})(\d\d\d\d)', r'var \1 = "\4-\3-\2"', raw_line )
    
    return kill_extra_spaces( raw_line )

translate_date_vars( "local begpart2date 01oct2017" )

'val begpart2date = "2017-10-01"'

### Translate "d( var )" to "to_date( '$date_var' )"

In [16]:
def translate_date_function( raw_line ):
    
    raw_line = re.sub( r" d\( `([A-Za-z0-9]{1,})' \) ", r" to_date( '$\1' ) ", raw_line )
    
    return kill_extra_spaces( raw_line )

translate_date_function( "( PIRL900 <= d( `qtrend' ) )" )

"( PIRL900 <= to_date( '$qtrend' ) )"

In [17]:
for line in stata_list:
    
    words = line.split( " " )
    if words[ 0 ] in translatable:
        
        line = normalize_parenthesis( line )
        line = normalize_plus_signs( line )
        line = normalize_equals( line )
        line = translate_conjuntions( line )
        line = translate_variable_declarations( line )
        line = translate_in_clause( line )
        line = translate_date_vars( line )
        line = translate_date_function( line )
        
        print( line )
        
    else:
        print( "// " + line )

// ********************************************************************************
// *************************QPR TEST AGGREGATION RULES*****************************
// ********************************************************************************
// cd "H:\Data Work\WIPS Test File"
local cdate: display %td-CY-m-D date( c( current_date ),"DMY" )
// copy "QPR_Template.xlsx" "StataQPROutput`cdate'.xlsx", replace
// set more off
// putexcel clear
// putexcel set "StataQPROutput`cdate'.xlsx", modify keepcellformat
// ************FUNDING STREAMS*****************************************************
val Adult = "( PIRL903 IN ( 1,2,3,4 ) )"
val DW = "( PIRL904 IN ( 1,2,3,4 ) OR PIRL909 = 1 )"
val WP = "( PIRL918 IN ( 1,2 ) )"
val Youth = "( PIRL905 IN ( 1,2,3,4 ) )"
// ************PARTICIPANT AND REPORTABLE INDIVIDUAL SPLITS************************
val partsplit = "( PIRL903 IN ( 1,2,3 ) OR PIRL904 IN ( 1,2,3 ) OR PIRL918 IN ( 1 ) OR PIRL905 IN ( 1,2,3 ) )"
val risplit = "( PIRL903 IN ( 4 

In [22]:
foo = stata_list[ 0 ]
print( foo )


********************************************************************************
